In [2]:
import torch
import torch.nn as nn
from dataclasses import dataclass

import requests
import unicodedata

from jaxtyping import Int, Float
from collections import Counter
import numpy as np

# Classes

In [3]:
@dataclass
class Config:
    d_model: int
    d_vocab: int
    d_hidden: int
    n_context: int
    n_layers: int

In [4]:
# class Embedding(nn.Module):
#     def __init__(self):
#         super().__init__()
    
#     def forward(self):
#         pass

class Attention(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        # self.W_qk = nn.Linear(config.d_model, config.d_vocab)
        self.bilinear = nn.Bilinear(config.d_model, config.d_model, config.n_context, bias=False)
        self.M = torch.triu(torch.ones((config.n_context, config.n_context)), diagonal=1)
        self.M = self.M.masked_fill(self.M.bool(), -torch.inf)
        self.second_matmult = nn.Linear(config.d_model, config.d_model, bias=False)
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        xwx = self.bilinear(x, x) # d_m x d_m
        x_masked = xwx+ self.M 
        x_softmaxed = self.softmax(x_masked)
        x_fin = x_softmaxed@x
        #multiply softmaxed by x
        #multiply that by wov
        x_fin = self.second_matmult(x_fin)
        return x_fin

class MLP(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.linear_up = nn.Linear(config.d_model, config.d_hidden)
        self.linear_down = nn.Linear(config.d_hidden, config.d_model)
    
    def forward(self, x):
        x = self.linear_up(x)
        x = torch.relu(x)
        x = self.linear_down(x)
        return x
    
class TransformerBlock(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config

        self.MLP = MLP(config=self.config)
        self.Attention = Attention(config=self.config)
    
    def forward(self, x):
        return x + self.Attention(x) + self.MLP(x)
    
class Transformer(nn.Module):
    def __init__(self, config:Config):
        self.transformerBlock = nn.ModuleList([TransformerBlock(config) for i in range(config.n_layers)])
        self.embedding = nn.Embedding(num_embeddings=config.d_vocab, embedding_dim=config.d_model)

    def forward(self, x):
        x = self.embedding(x)
        for i, l in enumerate(self.transformerBlock):
            x = self.transformerBlock[i](x)
        return x

$n_c$: Context window length

$d_m$: Model Dimension

$d_v$: Vocab Dimension

In [5]:
text_sample = "The quick brown fox jumped over the lazy dog."

# Tokenization Code

In [6]:
from pathlib import Path

def get_gutenberg_book(
	id: int | None = 84,
	data_temp: Path | str = "../data/gutenberg_data",
	remove_gutenberg_meta: bool = True,
) -> str:
	
	data_temp: Path = Path(data_temp)
	data_temp.mkdir(parents=True, exist_ok=True)
	
	url: str = f"https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt"
	data_path: Path = Path(data_temp) / f"{id}.txt"
	data: str
	# read from cache if it exists
	if data_path.exists():
		with open(data_path, 'r', encoding='utf-8') as file:
			data = file.read()
	else:
		# download if it doesn't exist
		response: requests.Response = requests.get(url)
		response.raise_for_status()  # Ensure that the download was successful
		data = response.text

		# save to cache
		with open(data_path, 'w', encoding='utf-8') as file:
			file.write(data)

	# remove header/footer
	if remove_gutenberg_meta:
		data = '***'.join(data.split('***')[2:])
		data = '***'.join(data.split('***')[:-1])
	
	return data

def get_many_books(
		ids: list[int],
		data_temp: Path | str = "../data/gutenberg_data",
	) -> list[str]:
	
	data: list[str] = []
	for id in ids:
		print(f"Getting book {id}...")
		item: str = get_gutenberg_book(id, data_temp)
		print(f"\t{len(item)} characters read")
		data.append(item)
	
	return data

In [7]:
def process_text(
	text: str,
	allowed_punctuation: str = "-.,;:!?()\"\\" + "".join(str(x) for x in range(10)),
	punctuation_convert: dict[str, str] = {'—': '-'},
) -> str:
	
	# replace some special characters which unicode won't normalize properly
	for char, replacement in punctuation_convert.items():
		text = text.replace(char, replacement)

	# if a line has ".jpg" in it, remove that line (this is specific to Don Quixote)
	text = '\n'.join(
		line 
		for line in text.split('\n')
		if '.jpg' not in line
	)

	# Normalize the string to decompose Unicode characters
	text = unicodedata.normalize('NFKD', text)

	# Encode to ASCII bytes, then decode back to string, ignoring errors
	text = text.encode('ascii', 'ignore').decode('ascii')

	# remove newlines and tabs
	text = text.replace('\n', ' ').replace('\t', ' ')


	# put spaces around allowed punctuation
	for char in allowed_punctuation:
		text = text.replace(char, f' {char} ')


	# remove leading and trailing spaces
	text = text.strip()

	# remove multiple spaces
	while '  ' in text:
		text = text.replace('  ', ' ')


	# remove all characters except (alphanumeric, allowed_punctuation, ' ')
	text = ''.join(
		(
			char 
			if (
				char.isalnum() 
				or char in allowed_punctuation 
				or char == ' '
			)
			else ' '
		)
		for char in text 
	)

	# convert to lowercase
	text = text.lower()

	text = text.strip()

	return text

In [8]:
def tokenize(
	text: str,
	process: bool = False,
) -> list[str]:
	if process:
		text = process_text(text)
	return text.split(' ')

In [9]:
# Getting books from Plato and Aristotle
DATA_RAW: list[str] = get_many_books([6762, 1497, 8438, 1600, 1656])
DATA: str = " ".join(process_text(x) for x in DATA_RAW)
DATA_TOKENIZED: list[str] = tokenize(DATA)

Getting book 6762...
	584611 characters read
Getting book 1497...
	1219052 characters read
Getting book 8438...
	648768 characters read
Getting book 1600...
	181248 characters read
Getting book 1656...
	87284 characters read


In [10]:
# sorted by frequency
VOCAB_FREQ: Counter[str] = Counter(DATA_TOKENIZED)
VOCAB_ARR: list[str] = [word for word, _ in VOCAB_FREQ.most_common()]
VOCAB_DICT: dict[str, int] = {word: i for i, word in enumerate(VOCAB_ARR)}

def encode(
	text: str | list[str],
) -> Int[np.ndarray, " n_tokens"]:
	if isinstance(text, str):
		text = tokenize(text)
	return np.array([VOCAB_DICT[word] for word in text])

def decode(
	encoded_text: Int[np.ndarray, " n_tokens"] | list[int],
) -> str:
	return ' '.join(VOCAB_ARR[i] for i in encoded_text)

DATA_ENCODED: Int[np.ndarray, " n_tokens"] = encode(DATA)

print(f"{DATA_ENCODED = }")
print(len(DATA_ENCODED))

DATA_ENCODED = array([1181,   25, 9326, ..., 4819, 4354, 1842], shape=(626081,))
626081


# Tests

In [11]:
d_model = 10
d_vocab = 10
d_hidden = 10
n_context = 5
n_layers = 10

x = torch.randn((n_context, d_model))

conf = Config(d_model, d_vocab, d_hidden, n_context, n_layers)
mlp = MLP(conf)
attention = Attention(conf)
Aoutput = attention(x)
print(Aoutput.shape)

output = mlp(x)
print(output)

torch.Size([5, 10])
tensor([[-1.5844e-01, -1.2614e-01, -1.5132e-01, -7.4821e-02, -3.3354e-02,
         -1.7954e-01,  5.4231e-03,  5.0504e-01,  1.9708e-01,  5.3856e-01],
        [-2.9039e-02, -4.2993e-01,  3.6675e-02, -8.0885e-02,  1.5491e-01,
          3.9740e-01,  1.4762e-01,  2.9777e-01,  4.1032e-01, -1.1207e-01],
        [-1.8447e-01, -5.5580e-02,  1.8991e-02,  2.4289e-01,  2.5611e-01,
          2.5013e-04, -2.4621e-01,  3.9945e-01,  1.0713e-02,  3.0392e-01],
        [-3.0789e-01, -1.8269e-01, -1.0287e-01,  2.2861e-01,  2.4110e-01,
         -1.2972e-01, -1.8224e-01,  4.0773e-01,  1.4991e-01,  3.6655e-01],
        [ 2.3969e-01,  3.0128e-01,  1.3705e-01,  2.0809e-01, -2.6409e-01,
          4.6971e-01, -2.3857e-01,  2.3207e-01, -3.3052e-01,  5.8728e-02]],
       grad_fn=<AddmmBackward0>)


c:\Users\Sawyer\Documents\decodingGPT\transformer\.venv\Lib\site-packages\torch\nn\modules\module.py:1776: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [12]:
# Transformer Block test

d_model = 10
d_vocab = len(VOCAB_DICT)
d_hidden = 10
n_context = 5
n_layers = 10

config = Config(
    d_model = d_model,
    d_vocab = d_vocab,
    d_hidden = d_hidden,
    n_context = n_context,
    n_layers = n_layers,
)

x = torch.randn((n_context, d_model))
conf = Config(d_model, d_vocab, d_hidden, n_context, n_layers)

tb = TransformerBlock(config)

output_x = tb(x)
output_x


tensor([[ 1.2841,  1.2585, -0.3046,  1.2458,  1.5591, -1.8320, -2.0312,  0.7599,
         -0.4450,  0.2922],
        [-1.0739,  2.1877, -0.6875,  1.9496, -0.1120,  1.7716, -3.3345, -1.2047,
         -0.1815, -0.4620],
        [ 1.3334,  1.3246,  0.6766, -0.3026, -0.3247,  0.9369, -1.8377, -0.6661,
         -1.1505, -0.4307],
        [ 1.0869,  0.6554,  0.4093, -0.9846,  1.3897,  0.2202, -0.2722, -0.9686,
          0.1981,  1.2969],
        [ 0.1931, -0.3004, -1.2248, -0.1256,  1.0818,  0.3864,  0.1961, -1.7655,
         -0.2540, -0.4553]], grad_fn=<AddBackward0>)

## Training Loop

In [ ]:
conf = Config(d_model = 10, 
              d_vocab = len(VOCAB_DICT), 
              d_hidden = 10, 
              n_context = 10, 
              n_layers = 2
              )


transformer = Transformer(config=conf)
output = transformer(DATA_ENCODED)